In [2]:
!pip install deepface opencv-python-headless tqdm matplotlib mediapipe

  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 126.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  At

In [1]:
# Importa o DeepFace para análise de emoções faciais
from deepface import DeepFace

# OpenCV para leitura e escrita de vídeo, além de manipulação de imagem
import cv2

# tqdm para barra de progresso durante o processamento do vídeo
from tqdm import tqdm

# Counter e deque para contagem de eventos e armazenamento histórico com tamanho fixo
from collections import Counter, deque

# MediaPipe para extração de pontos do corpo (pose)
import mediapipe as mp

# json para salvar o relatório final com estatísticas
import json

# Caminhos para o vídeo de entrada e saída
input_video_path = './Video1.mp4'
output_video_path = './Saida_Video1.mp4'

# Inicialização do MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

# Filas circulares para armazenar histórico recente de movimentos (aceno e dança)
hand_wave_history = deque(maxlen=5)
dancing_history = deque(maxlen=5)

# Função principal para classificar a atividade corporal com base nos landmarks da pose
def classify_activity(pose_landmarks):
    if not pose_landmarks:
        return "desconhecida"  # Nenhum ponto de corpo detectado

    # Verifica se um landmark é visível com confiança (> 0.5)
    def is_visible(lm): return lm and lm.visibility > 0.5

    lm = pose_landmarks.landmark

    # Retorna o landmark com segurança (caso não exista ou dê erro)
    def get_safe(name):
        try:
            return lm[mp_pose.PoseLandmark[name]]
        except:
            return None

    # Coleta os principais pontos do corpo
    l_sh = get_safe("LEFT_SHOULDER")
    r_sh = get_safe("RIGHT_SHOULDER")
    l_hp = get_safe("LEFT_HIP")
    r_hp = get_safe("RIGHT_HIP")
    l_kn = get_safe("LEFT_KNEE")
    r_kn = get_safe("RIGHT_KNEE")
    l_wr = get_safe("LEFT_WRIST")
    r_wr = get_safe("RIGHT_WRIST")
    l_an = get_safe("LEFT_ANKLE")
    r_an = get_safe("RIGHT_ANKLE")
    r_eye = get_safe("RIGHT_EYE")
    l_eye = get_safe("LEFT_EYE")
    r_elbow = get_safe("RIGHT_ELBOW")
    l_elbow = get_safe("LEFT_ELBOW")

    # 1. Detecta se a pessoa está deitada
    if is_visible(r_eye) and is_visible(r_sh) and is_visible(l_eye):
        if (r_sh.y < r_eye.y) and (r_eye.y < l_eye.y):
            return "deitado"

    # 2. Detecta aceno (com a mão esquerda visível e direita não visível, acima do ombro)
    if not is_visible(r_wr) and is_visible(l_wr) and is_visible(l_sh):
        is_waving = l_wr.y < l_sh.y
        hand_wave_history.append(is_waving)
        if hand_wave_history.count(True) >= 3:
            return "acenando"

    # 3. Detecta dança (mãos visíveis, mão esquerda acima do ombro por repetição)
    if is_visible(r_wr) and is_visible(l_wr) and is_visible(l_sh):
        is_dancing = l_wr.y < l_sh.y
        dancing_history.append(is_dancing)
        if dancing_history.count(True) >= 3:
            return "dancando"

    # 4. Detecta se a pessoa está de perfil (com base na posição do ombro e profundidade)
    if is_visible(r_sh):
        if (r_wr.x > r_elbow.x) or (r_sh.z < l_sh.z):
            return "perfil"

    # 5. Detecta se a pessoa está com os olhos visíveis — pode indicar careta ou sorriso
    if is_visible(r_eye) and is_visible(l_eye):
        return "sorriso/careta"

    return "desconhecida"

# Função principal que processa o vídeo e classifica emoções + atividades
def detect_emotions(video_path, output_path, resize_factor=0.5, frame_skip=3):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Erro ao abrir o vídeo.")
        return

    # Extrai metadados do vídeo
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_size = (int(width * resize_factor), int(height * resize_factor))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, frame_size)

    # Contadores e armazenamento de histórico
    emotion_counter = Counter()
    activity_counter = Counter()
    unknown_activity_count = 0
    analyzed_frame_count = 0

    frame_index = 0
    face_emotion_history = {}  # Histórico de emoções por face detectada
    previous_emotion = {}      # Última emoção detectada (para suavização)

    # Loop principal de processamento de cada frame
    for _ in tqdm(range(total_frames), desc="Processando vídeo"):
        ret, frame = cap.read()
        if not ret:
            break

        # Redimensiona o frame, se necessário
        if resize_factor != 1.0:
            frame = cv2.resize(frame, frame_size)

        frame_index += 1
        if frame_index % frame_skip != 0:
            out.write(frame)
            continue

        # Converte para RGB para o MediaPipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results_pose = pose.process(rgb_frame)

        # Classificação da atividade corporal
        # Estou classificando a atividade corporal da pessoa mais proeminente no
        # frame. Media Pipe Pose no modo single-person.
        atividade = classify_activity(results_pose.pose_landmarks)
        analyzed_frame_count += 1
        activity_counter[atividade] += 1
        if atividade == "desconhecida":
            unknown_activity_count += 1

        # Análise de emoções com DeepFace. Tentei utilizar o "opencv" mas não
        # detectava rostos laterais. Mais rápido e leve.
        # O algorítimo "retinaface" é mais robusto e preciso. Execução pesada.
        # Explorei resize_factor=0.5, reduzindo o tamanho do frame analisado e
        # também ajuste o frame_skip, reduzindo a quantidade de frames analisados.
        # Testei com 10, 5 e 3. No total o vídeo analisado possui 3326 frames.
        # Só ganhei performance na análise quando passei a utilizar a GPU T4 -
        # NVIDIA Tesla T4 - no Colabs.

        try:
            result = DeepFace.analyze(
                frame,
                actions=['emotion'],
                enforce_detection=False,
                detector_backend='retinaface'
            )
        except Exception as e:
            print(f"[Frame {frame_index}] Erro ao analisar: {e}")
            out.write(frame)
            continue

        # Processa cada rosto detectado
        if isinstance(result, list) and result:
            for face in result:
                try:
                    region = face.get('region', {})
                    x = max(0, region.get('x', 0))
                    y = max(0, region.get('y', 0))
                    w = min(region.get('w', 0), frame.shape[1] - x)
                    h = min(region.get('h', 0), frame.shape[0] - y)

                    dominant_emotion = face.get('dominant_emotion', None)
                    if dominant_emotion:
                        emotion_counter[dominant_emotion] += 1

                    # Percepções iniciais sobre a detecção das emoções:
                    # As emoções estavam oscilando muito frame a frame devido a
                    # pequenas variações de iluminação, ângulo ou ruído. O vídeo
                    # final apresentou um resultado visual confuso e inconstante.

                    # Utilizei a técnica de suavização de emoção, assim, a emoção
                    # exibida na tela é mais estável, coerente e agradável ao usuário.

                    # Histórico da face (usando coordenadas como ID simplificado)
                    face_id = (x, y, w, h)
                    if face_id not in face_emotion_history:
                        face_emotion_history[face_id] = deque(maxlen=10)

                    face_emotion_history[face_id].append(dominant_emotion)

                    # Suavização de emoção (filtra oscilações)
                    emotion_freq = Counter(face_emotion_history[face_id])
                    most_common_emotion, count = emotion_freq.most_common(1)[0]
                    smoothed_emotion = most_common_emotion if count >= 5 else previous_emotion.get(face_id, most_common_emotion)
                    previous_emotion[face_id] = smoothed_emotion

                    # Desenha retângulo e texto com emoção e atividade
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    text_y = y - 10 if y - 10 > 20 else y + h + 20
                    cv2.putText(frame, smoothed_emotion, (x, text_y),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
                    cv2.putText(frame, atividade, (x, text_y + 25),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (36, 255, 12), 2)
                except Exception as e:
                    print(f"Erro ao processar face: {e}")
                    continue

        out.write(frame)

    cap.release()
    out.release()

    # Gera e salva relatório final em JSON
    report = {
        "Total de frames analisados": analyzed_frame_count,
        "Total de anomalias (atividade 'desconhecida')": unknown_activity_count,
        "Distribuição de emoções": dict(emotion_counter),
        "Distribuição de atividades": dict(activity_counter)
    }

    with open("relatorio_analise_video.json", "w", encoding="utf-8") as f:
        json.dump(report, f, indent=4, ensure_ascii=False)

    print("Relatório salvo em: relatorio_analise_video.json")


# Inicia o processamento do vídeo
detect_emotions(input_video_path, output_video_path)


Processando vídeo:   0%|          | 0/3326 [00:00<?, ?it/s]

25-07-27 23:32:59 - retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5

  0%|          | 0.00/119M [00:00<?, ?B/s]
  9%|▉         | 11.0M/119M [00:00<00:02, 39.3MB/s]
 18%|█▊        | 21.5M/119M [00:00<00:02, 40.1MB/s]
 27%|██▋       | 32.0M/119M [00:00<00:02, 40.0MB/s]
 36%|███▌      | 42.5M/119M [00:01<00:01, 40.4MB/s]
 45%|████▍     | 53.0M/119M [00:01<00:01, 41.3MB/s]
 53%|█████▎    | 63.4M/119M [00:01<00:01, 41.6MB/s]
 62%|██████▏   | 73.9M/119M [00:01<00:01, 41.1MB/s]
 71%|███████   | 84.4M/119M [00:02<00:00, 41.8MB/s]
 80%|███████▉  | 94.9M/119M [00:02<00:00, 41.2MB/s]
 89%|████████▉ | 105M/119M [00:02<00:00, 40.1MB/s] 
100%|██████████| 119M/119M [00:02<00:00, 41.2MB/s]


25-07-27 23:33:09 - facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5

100%|██████████| 5.98M/5.98M [00:00<00:00, 152MB/s]
Processando vídeo: 100%|██████████| 3326/3326 [06:41<00:00,  8.28it/s]

Relatório salvo em: relatorio_analise_video.json
